**Update - 23rd Dec, 2021**

We have completed the TF-JAX tutorials series. 10 notebooks that covers every fundamental aspect of both TensorFlow and JAX. Here are the links to the notebooks along with the Github repo details:

### TensorFlow Notebooks:

* [TF_JAX_Tutorials - Part 1](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part1)
* [TF_JAX_Tutorials - Part 2](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part2)
* [TF_JAX_Tutorials - Part 3](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part3)

### JAX Notebooks:

* [TF_JAX_Tutorials - Part 4 (JAX and DeviceArray)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-4-jax-and-devicearray)
* [TF_JAX_Tutorials - Part 5 (Pure Functions in JAX)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-5-pure-functions-in-jax/)
* [TF_JAX_Tutorials - Part 6 (PRNG in JAX)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-6-prng-in-jax/)
* [TF_JAX_Tutorials - Part 7 (JIT in JAX)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-7-jit-in-jax)
* [TF_JAX_Tutorials - Part 8 (Vmap and Pmap)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-8-vmap-pmap)
* [TF_JAX_Tutorials - Part 9 (Autodiff in JAX)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-9-autodiff-in-jax)
* [TF_JAX_Tutorials - Part 10 (Pytrees in JAX)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-10-pytrees-in-jax)

### Github Repo with all notebooks in one place
https://github.com/AakashKumarNain/TF_JAX_tutorials

---

<img src="https://i.ytimg.com/vi/yjprpOoH5c8/maxresdefault.jpg" width="300" height="300" align="center"/>

Welcome to another TensorFlow/JAX tutorial. As I said in [Part1](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part1), these tutorials are meant for everyone (from novice to advanced). This tutorial is in continuation of the last part. If you haven't gone through [TF-JAX Tutorials - Part1](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part1), I highly recommend reading it before diving into this one. Let's start!


**Reminder** These tutorials are in accordance with this format:
1. TF Fundamentals (2-3 notebooks)
2. JAX Fundamentals (2-3 notebooks)
3. Advanced TF (2-3 notebooks)
4. Advanced JAX (2-3 notebooks)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)
%config IPCompleter.use_jedi = False

In the last notebook, we read about the following:<br>
1. Tensors
2. Different ways of creating a tensor in TF
3. Immutability in TF tensors
4. Special operations like `tf.gather` and `tf.scatter`

In this tutorial, we will be looking at another building block. 

## Variables

A `Variable` is a "special" kind of tensor. It is used to represent or store the mutable state. A `tf.Variable` represents a tensor whose value can be changed by running ops on it. Think of a situation where you would use a `Variable` object? **Weights** of neural networks is one of the best examples of usages of Variables. 

We will first see how Variable objets are created, and then we will look into the properties and some of the gotchas

### Creating a Variable

Good news! There is only one method that creates a `Variable` object: `tf.Variable(..)`

In [2]:
# Variables with an integer value of 2 as initial value
x = tf.Variable(2)
x

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>

In [3]:
# Nested list as initial value
y = tf.Variable([[2, 3]], dtype=tf.int32)
y
print(y.shape)

(1, 2)


In [4]:
# Tuples also work but beware it isn't the same as a nested list.
# Check the difference between the current output and the previous cell output
y = tf.Variable(((2, 3)), dtype=tf.int32)
y
print(y.shape)

(2,)


In [5]:
# You can even pass a tensor object as an initial value
t = tf.constant([1, 2,], dtype=tf.int32)
z = tf.Variable(t)
z
print(z.shape)

(2,)


In [6]:
# An interesting thing to note. 
# You can't change the values of the tensor `t` in the above example
# but you can change the values of the variable created using it

# This won't work
try:
    t[0] = 1
except Exception as ex:
    print(type(ex).__name__, ex)
    
# This also won't work
try:
    z[0] = 10
except Exception as ex:
    print(type(ex).__name__, ex)
    
# This works though
print("\nOriginal variable: ", z)
z[0].assign(5)
print("Updated variable: ", z)
z.assign([6,6])
print("Updated variable: ", z)

TypeError 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
TypeError 'ResourceVariable' object does not support item assignment

Original variable:  <tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>
Updated variable:  <tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([5, 2], dtype=int32)>
Updated variable:  <tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([6, 6], dtype=int32)>


**A few things to note:**

1. You can create a variable by passing an initial value which can be a `Tensor`, or Python object convertible to a `Tensor` 
2. The tensor object that you are passing is immutable but the variable created using it is mutable
3. Variable is a `special` kind of tensor but the underlying data structure for both `tensors` and `variables` is `tf.Tensor`
4. Since the data structure is the same, most of the properties for the two are same. We will take an example of this in a moment.
5. Direct assignment (like z[0]=5) doesn't work with `tf.Variable` as well. For changing the values, you need to call the methods like `assign(...)`, `assign_add(...)` or `assign_sub(...)` 
6. Any Variable has the same lifecycle as any other Python object. When there are no references to a variable it is automatically deallocated.

In [7]:
# Most of the properties that we saw for tensors in part1 are the same for variables
print(f"Shape of variable : {z.shape}")
print(f"Another method to obtain the shape using `tf.shape(..)`: {tf.shape(z)}")

print(f"dtype of the variable: {z.dtype}")
print(f"Total size of the variable: {tf.size(z)}")
print(f"Values of the variable: {z.numpy()}")

Shape of variable : (2,)
Another method to obtain the shape using `tf.shape(..)`: [2]
dtype of the variable: <dtype: 'int32'>
Total size of the variable: 2
Values of the variable: [6 6]


In [8]:
#  This doesn't work though
try:
    print(f"Rank: {z.ndim}")
except Exception as ex:
    print(type(ex).__name__, ex)
    
# That does work
print(f"ndim for z= {z.numpy().ndim}")

AttributeError 'ResourceVariable' object has no attribute 'ndim'
ndim for z= 1


In [9]:
# Crap! How to find out the no of dimensions then?
print(f"Rank: {tf.shape(z)} or like this {z.shape}")

Rank: [2] or like this (2,)


In [10]:
# Whatever operator overloading is available for a Tensor, is also available for a Variable
# We have a tensor `t` and a varibale `z`. 

t = tf.constant([1, 2,], dtype=tf.int32)
z = tf.Variable(t)
print("Tensor t: ", t)
print("Variable z: ", z)

print("\nThis works: ", (t+5))
print("So does this: ", (z +5))

print(f"Another example just for demonstration: {(t*5).numpy()}, {(z*5).numpy()}")

Tensor t:  tf.Tensor([1 2], shape=(2,), dtype=int32)
Variable z:  <tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>

This works:  tf.Tensor([6 7], shape=(2,), dtype=int32)
So does this:  tf.Tensor([6 7], shape=(2,), dtype=int32)
Another example just for demonstration: [ 5 10], [ 5 10]


In [11]:
# Gather works as well
tf.gather(z, indices=[0])

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [12]:
# Here is another interesting difference between the properties of 
# a tensor and a variable

try:
    print("Is variable z trainable? ", z.trainable)
    print("Is tensor t trainable? ", t.trainable)
except Exception as ex:
    print(type(ex).__name__, ex)

Is variable z trainable?  True
AttributeError 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'trainable'


Let's talk about a bit why `trainable` is an interesting property when it comes to a Variable object.
1. Any variable is tracked by Gradient tape (if it's in the scope) automatically unless it isn't trainable
2. Any variable that is defined within the scope of a class that inherits `tf.Module` is tracked automatically and can be collected via the `trainable_variables`, `variables`, or `submodule` property (More on this in the future notebooks)
3. Sometimes we don't want the gradients for a certain Variable. In that case, we can turn off the tracking by setting `trainable=False`. One example for this can be a counter

In [13]:
x = tf.Variable(2.0, name="x")
y = tf.Variable(4.0, trainable=False, name="y")
z = tf.Variable(6.0, name="z")

with tf.GradientTape() as tape:
    x = x + 2
    y = y + 5
    z = 2*z*z

print([variable.name for variable in tape.watched_variables()])

['x:0', 'z:0']


The biggest advantage with a variable is that the memory can be reused. You can modify the values without creating a new one, though there are certain things to keep in mind. Check this out

In [14]:
# Create a variable instance
z = tf.Variable([1, 2], dtype=tf.int32, name="z")
print(f"Variable {z.name}: ", z)

# Can we change the dtype while changing the values?
try:
    z.assign([1.0, 2.0])
except Exception as ex:
    print("\nOh dear...what have you done!")
    print(type(ex).__name__, ex)
    
# Can we change the shape while assigning a new value?
try:
    z.assign([1, 2, 3])
except Exception as ex:
    print("\nAre you thinking clearly?")
    print(type(ex).__name__, ex)
    
# A way to create variable with an arbitrary shape
x = tf.Variable(5, dtype=tf.int32, shape=tf.TensorShape(None), name="x")
print("\nOriginal Variable x: ", x)

# Assign a proper value with a defined shape
x.assign([1, 2, 3])
print("Modified Variable x: ", x)

# Try assigning a value with a diff shape now.
try:
    x.assign([[1, 2, 3], [4, 5, 6]])
    print("\nThis works!!")
    print("Variable value modified with a diff shape: ", x)
except Exception as ex:
    print("\nDid you forget what we just learned?")
    print(type(ex).__name__, ex)

Variable z:0:  <tf.Variable 'z:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>

Oh dear...what have you done!
TypeError Cannot convert [1.0, 2.0] to EagerTensor of dtype int32

Are you thinking clearly?
ValueError Cannot assign to variable z:0 due to variable shape (2,) and value shape (3,) are incompatible

Original Variable x:  <tf.Variable 'x:0' shape=<unknown> dtype=int32, numpy=5>
Modified Variable x:  <tf.Variable 'x:0' shape=<unknown> dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

This works!!
Variable value modified with a diff shape:  <tf.Variable 'x:0' shape=<unknown> dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>


That's it for part 2! I hope you enjoyed reading it. We will be looking at other things k like `GradientTape` in the next tutorial!<br>

**References**:
1. https://www.tensorflow.org/guide/variable
2. https://keras.io/getting_started/intro_to_keras_for_researchers/


Update:

[TF_JAX_Tutorials - Part 3](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part3)<br>
[TF_JAX_Tutorials - Part 4 (JAX and DeviceArray)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-4-jax-and-devicearray/)<br>
[TF_JAX_Tutorials - Part 5 (Pure Functions)](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part-5-pure-functions-in-jax)